In [3]:
import pandas as pd
import os

In [ ]:
nested_path = "/home/shu/projects/stress-strain-prediction/Clean_Data/"   
csv_path = os.path.join(nested_path, "all_steel_curves.csv")

df = pd.read_csv(csv_path)
df.head()

FileNotFoundError: [Errno 2] No such file or directory: '/home/shu/Projects/ss_curve/Clean_Data/all_steel_curves.csv'

In [3]:
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 291522 entries, 0 to 291521
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   stress       291522 non-null  float64
 1   strain       291522 non-null  float64
 2   steel_grade  291522 non-null  object 
 3   file_name    291522 non-null  object 
dtypes: float64(2), object(2)
memory usage: 8.9+ MB


,stress,strain
count,291522.000000,291522.000000
mean,65.011844,0.006496
std,374.151840,0.028607
min,-970.754480,-0.079963
25%,-309.000000,-0.007158
50%,131.828581,0.001300
75%,393.868966,0.011972
max,958.571166,0.309122


In [4]:
df = df[~((df['stress'] == 0) & (df['strain'] == 0) & df.duplicated())]
df.head()

,stress,strain,steel_grade,file_name
0,0.0,0.0,HYP400,HYP_t27_2inc.csv
20,0.1,0.0,HYP400,HYP_t27_2inc.csv
21,0.3,0.0,HYP400,HYP_t27_2inc.csv
22,0.7,0.0,HYP400,HYP_t27_2inc.csv
23,2.2,0.0,HYP400,HYP_t27_2inc.csv


In [5]:
df['stress_norm'] = df.groupby('steel_grade')['stress'].transform(lambda x: x / x.max())
df['strain_norm'] = df.groupby('steel_grade')['strain'].transform(lambda x: x / x.max())
df.head()

,stress,strain,steel_grade,file_name,stress_norm,strain_norm
0,0.0,0.0,HYP400,HYP_t27_2inc.csv,0.000000,0.0
20,0.1,0.0,HYP400,HYP_t27_2inc.csv,0.000155,0.0
21,0.3,0.0,HYP400,HYP_t27_2inc.csv,0.000464,0.0
22,0.7,0.0,HYP400,HYP_t27_2inc.csv,0.001082,0.0
23,2.2,0.0,HYP400,HYP_t27_2inc.csv,0.003399,0.0


In [1]:
curve_vectors = df.groupby('file_name').apply(
    lambda g: g[['strain_norm','stress_norm']].values.tolist()
)

curve_vectors.head()


NameError: name 'df' is not defined

In [ ]:
meta_path = os.path.join(nested_path, "db_tag_clean_data_map.csv")
df_meta = pd.read_csv(meta_path)
df_merged = df.merge(df_meta, on="steel_grade", how="left")
df_merged.head()

In [ ]:
meta_numeric = [c for c in df_meta.columns if df_meta[c].dtype in ['float64','int64']]

for col in meta_numeric:
    mean = df_meta[col].mean()
    std = df_meta[col].std()
    if std != 0:
        df_meta[col] = (df_meta[col] - mean) / std

df_merged = df.merge(df_meta, on="steel_grade", how="left")
df_merged.head()